In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam

# Assume data is already loaded into a pandas DataFrame
df = pd.read_csv('ball_by_ball_data.csv')

# Preprocessing
def preprocess_data(df):
    # Label encoding for categorical variables
    label_encoder = LabelEncoder()
    df['batsman'] = label_encoder.fit_transform(df['batsman'])
    df['bowler'] = label_encoder.fit_transform(df['bowler'])
    df['batting_team'] = label_encoder.fit_transform(df['batting_team'])
    df['bowling_team'] = label_encoder.fit_transform(df['bowling_team'])
    
    # Normalize the numerical features
    scaler = MinMaxScaler()
    df[['inning', 'over', 'ball', 'batsman_runs', 'extra_runs', 'total_runs']] = scaler.fit_transform(df[['inning', 'over', 'ball', 'batsman_runs', 'extra_runs', 'total_runs']])
    
    return df

df = preprocess_data(df)

# Sequence creation
def create_sequences(df, sequence_length):
    sequences = []
    for i in range(len(df) - sequence_length):
        sequence = df.iloc[i:i+sequence_length].values
        sequences.append(sequence)
    return np.array(sequences)

sequence_length = 10  # Choose a sequence length based on your data
sequences = create_sequences(df, sequence_length)

# Split data into input features (X) and target variable (y)
X = sequences[:, :-1, :]
y = sequences[:, -1, 6]  # Assuming the target is 'total_runs' for prediction

# Reshape X for LSTM/GRU
X = np.reshape(X, (X.shape[0], X.shape[1], X.shape[2]))

# Model architecture
model = Sequential()
model.add(LSTM(64, return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
model.add(GRU(32, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='linear'))  # Regression output for predicting runs

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

# Train the model
model.fit(X, y, epochs=20, batch_size=64, validation_split=0.2)

# Save the model
model.save('cricket_score_prediction_model.h5')


C:\Users\shoya\AppData\Local\Temp\ipykernel_4208\422006112.py:10: DtypeWarning: Columns (12,13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('ball_by_ball_data.csv')


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).

In [ ]:
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Sample players list
players_list = [
    {"Player Name": "SC Ganguly", "Team": "Kolkata Knight Riders"},
    {"Player Name": "BB McCullum", "Team": "Kolkata Knight Riders"},
    {"Player Name": "RT Ponting", "Team": "Kolkata Knight Riders"},
    # ... (more players)
    {"Player Name": "V Kohli", "Team": "Royal Challengers Bangalore"},
]

# Assuming you have pre-trained encoders and a trained model
player_encoder = LabelEncoder()  # Load or train on all player names
team_encoder = LabelEncoder()    # Load or train on all team names

# Load the trained model
from tensorflow.keras.models import load_model
model = load_model('path_to_your_lstm_gru_model.h5')

# Prepare data for prediction
player_names = [p['Player Name'] for p in players_list]
teams = [p['Team'] for p in players_list]

# Encode the player names and teams
encoded_players = player_encoder.transform(player_names)
encoded_teams = team_encoder.transform(teams)

# Combine the encoded features
features = np.array([encoded_players, encoded_teams]).T

# Pad sequences if needed (assuming input sequences should be of length 5)
features_padded = pad_sequences([features], maxlen=5, padding='post', truncating='post')

# Make predictions
predicted_points = model.predict(features_padded)

# Display the predictions
for player, points in zip(players_list, predicted_points[0]):
    print(f"Predicted Fantasy Points for {player['Player Name']} ({player['Team']}): {points:.2f}")


In [4]:
l = [[3, 0], [4, 1], [4, 3], [3, 4]]
l[-1][0]
val, idx = l.pop()

print(f'Value: {val}, Index: {idx}')

3